In [1]:
import pandas as pd
import requests, time, re

In [2]:
log = pd.read_excel('iris_clowder_log_with_metadata_20220729_jwilli29.xlsx')
iris = pd.read_excel('iris_assessment_table_unique_names_20220721.xlsx')

In [24]:
for i in range(3, len(log)):
    # ignore thumbs.db
    if i == 1101:
        continue
    # get document_name from origPath 
    path = log.loc[i, 'origPath']
    log.loc[i, 'document_name'] = path.split('\\')[-1]
    
    names = iris['Chemical Name'].tolist()
    # check the structure of the document name and extract it accordingly
    if bool(re.fullmatch(r'.*.pdf', path)) == True:
        name = path.split('_')[1]
    else:
        name = path.split('\\')[-1][:-4]
        
    # add the chemical name to log
    log.loc[i, 'chemical_name'] = name
    # find the CAS number in iris and add that to log
    log.loc[i, 'casrn'] = iris.loc[names.index(name), 'CASRN']
    # grab the URL from iris and add it to log
    if bool(re.search('_summary_', path)):
        log.loc[i, 'URL'] = iris.loc[names.index(name), 'IRIS Summary URL']
    else:
        log.loc[i, 'URL'] = iris.loc[names.index(name), 'Chemical URL']

log['source_name'] = 'IRIS' 
log['clowder_id'] = log['uuid']

In [25]:
log.to_excel('iris_document_map_2022_08_01.xlsx', index = False)